# 3 - Construindo um LLM do Zero: criando significado com Embeddings

Este é o **terceiro** de uma série de oito artigos que podem ser encontrados no meu medium. Acesse o primeiro artigo da série aqui: [Construindo um LLM: entendendo os Grandes Modelos de Linguagem](https://blog.zfab.me/construindo-um-llm-entendendo-os-grandes-modelos-de-linguagem-b37884219eaa)

--

Você sabia que as palavras podem ser organizadas em um espaço multidimensional que revela suas relações semânticas e contextuais?

Imagine uma biblioteca onde nenhum livro possui título ou categoria visível. Para organizá-la, analisamos características como peso, número de páginas e autor, posicionando os livros nas prateleiras de acordo com suas similaridades. É exatamente isso que **embeddings** fazem, transformam palavras, frases ou tokens em representações dentro de um espaço multidimensional — e é esse universo que vamos explorar.

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import torch
import torch.nn as nn

## **Representação Vetorial dos Tokens**

Anteriormente, através do processo de tokenização e do vocabulário, convertemos palavras em identificadores numéricos únicos (IDs). Porém, usar esses IDs diretamente como entrada não é eficaz. Por exemplo, em um vocabulário onde "gato" tem ID 1, "elefante" ID 2 e "cachorro" ID 3, o modelo poderia interpretar erroneamente esses números como valores ordinais, sugerindo que "cachorro" (ID 3) é maior ou tem uma relação quantitativa com "elefante" (ID 2).

Os IDs são identificadores arbitrários que, sozinhos, não carregam o verdadeiro significado dos tokens que representam. É como um CPF: ele não representa quem você é, apenas serve como um identificador numérico. Para informar melhor ao modelo o que cada ID significa, utilizamos vetores – listas de números – que nos oferecem maior flexibilidade na representação dos tokens. Em vez de um único número, trabalharemos com uma lista deles.

Surge então um novo desafio: como criar vetores que não sejam apenas listas arbitrárias de números, mas que realmente transmitam o significado das palavras? Para resolver essa questão, foram desenvolvidas várias técnicas que transformam palavras em representações numéricas mais ricas e significativas.

Uma abordagem fundamental para transformar tokens em vetores é conhecida como **one-hot encoding**. Esta técnica, embora simples, ilustra bem o conceito de representação vetorial: cada palavra é convertida em um vetor binário onde apenas a posição correspondente ao seu ID recebe o valor 1, enquanto todas as outras posições são preenchidas com 0. Por exemplo, em um vocabulário de 1000 palavras, a palavra "gato" poderia ser representada por um vetor de 1000 posições, com um único 1 na posição do seu ID e zeros em todas as outras.

In [3]:
vocab_onehot = {
    "O": 1,
    "rato": 2,
    "comeu": 3,
    "roeu": 4,
    "roupa": 5,
    "a": 6,
    "rainha": 7,
    "do": 8,
    "roma": 9,
    "de": 10,
    "Roma": 11,
    "Rei": 12
}

vector_onehot = {
    k: [1 if i == v else 0 for i in range(1, len(vocab_onehot))] for k, v in vocab_onehot.items()
}

phrase = "O rato roeu a roupa do Rei de Roma"
tokens = phrase.split(" ")

print("Frase:", phrase)
print("Tokens IDs:", [vocab_onehot[t] for t in tokens])
print("Tokens one-hot:", [vector_onehot[t] for t in tokens])

Frase: O rato roeu a roupa do Rei de Roma
Tokens IDs: [1, 2, 4, 6, 5, 8, 12, 10, 11]
Tokens one-hot: [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [4]:
vector_onehot

{'O': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'rato': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'comeu': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'roeu': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'roupa': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'a': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'rainha': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'do': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 'roma': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'de': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'Roma': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 'Rei': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Embora o one-hot encoding represente um avanço em relação ao uso direto de IDs numéricos, pois permite uma representação mais estruturada das palavras, esta técnica ainda apresenta limitações significativas que a tornam inadequada para processamento de linguagem natural moderno:

1. **Alta Dimensionalidade**: Em vocabulários extensos, que podem conter dezenas ou centenas de milhares de palavras, os vetores tornam-se extremamente longos e esparsos, resultando em uso ineficiente de memória e poder computacional.
2. **Falta de Relacionamento Semântico**: Assim como os IDs numéricos simples, o one-hot encoding não captura relações semânticas entre palavras. Por exemplo, "gato" e "felino" são representados por vetores completamente diferentes, sem qualquer indicação de sua proximidade conceitual.
3. **Vetores Esparsos**: Com apenas um valor não-zero em cada vetor, as operações matemáticas tornam-se computacionalmente custosas e ineficientes, especialmente em redes neurais modernas que precisam processar grandes volumes de texto.

Outras abordagens tradicionais de vetorização, como Bag of Words (BoW) e Term Frequency-Inverse Document Frequency (TF-IDF), tentam resolver algumas dessas questões ao criar representações para frases e documentos inteiros. Contudo, estas técnicas ainda sofrem com problemas de esparsidade, ausência de relações semânticas e, crucialmente, não preservam a ordem das palavras no texto.

Felizmente, essas limitações fundamentais são superadas, ou pelo menos significativamente reduzidas, através do uso de Embeddings

## **Embeddings**

Para superar essas limitações, surgiram os embeddings — vetores densos e multidimensionais que capturam propriedades semânticas e contextuais das palavras.

Para entender melhor, considere uma lista de animais: "gato", "cachorro", "cobra", "águia" e "elefante". Cada palavra pode ser descrita por características numéricas:

![Representação Vetorial](../assets/3-REPRESENTACAO_VETORIAL.png)

Podemos representar **Aranha** com o vetor [8, 0, 0.02] e **Cachorro** com o vetor [4, 1, 7]. Esta representação permite que o modelo identifique similaridades entre "gato" e "cachorro" através de suas características compartilhadas, ao mesmo tempo que reconhece as diferenças contextuais entre eles e "cobra".

In [5]:
# Dicionário com palavras e vetores de 3 posições
dict_animals = {
    "Gato": [4, 1, 4],          # 4 patas, mamífero, 4 kg
    "Cachorro": [4, 1, 7],      # 4 patas, mamífero, 7 kg
    "Águia": [2, 0, 5],         # 2 patas, não mamífero, 5 kg
    "Aranha": [8, 0, 0.02],     # 8 patas, não mamífero, 0.02 kg
    "Papagaio": [2, 0, 1],      # 2 patas, não mamífero, 1 kg
    "Cobra": [0, 0, 15],        # 0 patas, não mamífero, 15 kg
}

words = list(dict_animals.keys())
vectors = list(dict_animals.values())
x = [vec[0] for vec in vectors]
y = [vec[1] for vec in vectors]
z = [vec[2] for vec in vectors]

fig = px.scatter_3d(x=x, y=y, z=z, color=words, text=words)

fig.update_traces(
    marker=dict(size=8), 
    textposition='top left',
    textfont_size=12,
)

fig.update_layout(
    title='Embedding de Animais',
    scene = dict(
        xaxis_title='Patas',
        yaxis_title='Mamífero',
        zaxis_title='Peso'
    ),
    scene_aspectratio=dict(x=1.2, y=1, z=.7),
    scene_camera_eye=dict(x=1.5, y=1.1, z=1.5),
    margin=dict(l=20, r=20, b=50, t=50),
    showlegend=False,
    width=600,
    height=500,
)
fig.show()

Ao plotarmos esses animais em um espaço onde cada eixo representa uma propriedade, notamos que animais similares, como águia e papagaio ou cachorro e gato, ficam próximos uns aos outros, enquanto aranha e cobra se posicionam distantes.

Agora imagine que, em vez de animais, temos palavras (ou tokens) e, em vez de 3 características, temos centenas ou milhares. Se pudéssemos visualizar esse espaço (conhecido como espaço latente ou espaço de embedding), veríamos que palavras semelhantes como "rei" e "rainha" ficariam próximas, assim como "Brasília" e "Brasil". Notaríamos também que "rei" e "rainha" mantêm a mesma relação que "homem" e "mulher", preservando distâncias equivalentes.

Os embeddings são, essencialmente, representações vetoriais de elementos não numéricos (como palavras ou frases) em espaços multidimensionais. Em vez de definirmos manualmente as features, como fizemos com os animais (número de patas, peso, etc.), os valores e suas propriedades são aprendidos automaticamente durante o treinamento.

Para criarmos embeddings no PyTorch, basta usarmos o nn.Embedding, informando o tamanho do vocabulário e a dimensão do embedding (número de features). A camada de embedding funciona como uma grande tabela que relaciona cada ID ao seu vetor correspondente.

In [6]:
torch.nn.Embedding(
  num_embeddings=5000,    # Tamanho do Vocabulário 
  embedding_dim=300       # Tamanho do Embedding (número de features)
)

Embedding(5000, 300)

Agora, vamos criar uma camada de Embedding e aplicá-la em um exemplo prático. A partir deste ponto, passaremos a chamar os vetores de tensores. Embora o conceito fundamental seja o mesmo, no contexto de Deep Learning é convenção nos referirmos a vetores e matrizes como tensores.

In [94]:
VOCAB = {
    "O": 1,
    "rato": 2,
    "comeu": 3,
    "roeu": 4,
    "roupa": 5,
    "a": 6,
    "rainha": 7,
    "do": 8,
    "roma": 9,
    "Roma": 11,
    "de": 10,
    "Rei": 12
}

VOCAB_SIZE = len(VOCAB) # Tamanho do nosso vocabulário
EMBEDDING_DIM = 10 # Dimensão dos vetores de embedding

torch.manual_seed(42)
embedding_layer = nn.Embedding(num_embeddings=VOCAB_SIZE+1, embedding_dim=EMBEDDING_DIM)

phrase = "O rato roeu a roupa do Rei de Roma"
tokens = phrase.split(" ")

input_ids = torch.tensor([VOCAB[t] for t in tokens])  # IDs dos tokens
vectors = embedding_layer(input_ids)

print("Frase:", phrase)
print("Tokens IDs:", [VOCAB[t] for t in tokens])
print("Tokens embeddings:", vectors)

Frase: O rato roeu a roupa do Rei de Roma
Tokens IDs: [1, 2, 4, 6, 5, 8, 12, 10, 11]
Tokens embeddings: tensor([[-3.9248e-01, -1.4036e+00, -7.2788e-01, -5.5943e-01, -7.6884e-01,
          7.6245e-01,  1.6423e+00, -1.5960e-01, -4.9740e-01,  4.3959e-01],
        [-7.5813e-01,  1.0783e+00,  8.0080e-01,  1.6806e+00,  1.2791e+00,
          1.2964e+00,  6.1047e-01,  1.3347e+00, -2.3162e-01,  4.1759e-02],
        [-1.5576e+00,  9.9564e-01, -8.7979e-01, -6.0114e-01, -1.2742e+00,
          2.1228e+00, -1.2347e+00, -4.8791e-01, -9.1382e-01, -6.5814e-01],
        [-9.7807e-02,  1.8446e+00, -1.1845e+00,  1.3835e+00,  1.4451e+00,
          8.5641e-01,  2.2181e+00,  5.2317e-01,  3.4665e-01, -1.9733e-01],
        [ 7.8024e-02,  5.2581e-01, -4.8799e-01,  1.1914e+00, -8.1401e-01,
         -7.3599e-01, -1.4032e+00,  3.6004e-02, -6.3477e-02,  6.7561e-01],
        [ 1.0868e-02, -3.3874e-01, -1.3407e+00, -5.8537e-01,  5.3619e-01,
          5.2462e-01,  1.1412e+00,  5.1644e-02,  7.4395e-01, -4.8158e-01],
  

A partir de uma única frase, realizamos um processo básico de tokenização, convertemos os tokens em IDs usando um vocabulário e obtivemos seus respectivos tensores na camada de Embedding.

É importante destacar que, neste estágio, esses tensores ainda **não têm significado** — são apenas números aleatórios gerados, pois a camada de Embedding ainda não foi treinada. Assim, os valores que deveriam representar o significado de cada token ainda não foram aprendidos.

### **Dimensionalidade e Eficiência**

A escolha da dimensionalidade dos embeddings é crucial. Dimensões menores processam mais rapidamente, mas podem não capturar nuances suficientes. Já dimensões muito grandes aumentam o custo computacional e o consumo de memória.

> O modelo GPT-3 da OpenAI utiliza embeddings com 12.288 dimensões.
> 

O tamanho do vocabulário é fundamental. Vocabulários grandes demandam mais memória para processar embeddings, mas permitem capturar maior diversidade linguística.

A dimensionalidade também afeta diretamente a capacidade do modelo de capturar relações complexas. O tamanho da camada de embedding é uma decisão crítica que influencia tanto o desempenho quanto o custo computacional do modelo.

## **Codificação Posicional: Incorporando Ordem**

Modelos baseados em RNNs possuem um viés indutivo natural — suas características estruturais já incluem pressupostos sobre a organização dos dados. Como os tokens são processados sequencialmente, a posição das palavras na frase é naturalmente incorporada ao fluxo de cálculos, eliminando a necessidade de informar explicitamente a ordem temporal do texto.

Já os Transformers não possuem esse viés indutivo intrínseco. Por processarem todos os tokens simultaneamente, sua arquitetura não captura naturalmente a posição das palavras no texto. Isso representa um desafio significativo, pois a ordem das palavras pode alterar completamente o significado de uma frase — "o cachorro mordeu o homem" tem um significado muito diferente de "o homem mordeu o cachorro".

Para que os Transformers compreendam a estrutura do texto, precisamos adicionar informações explícitas sobre a posição de cada token. Isso é feito através do positional encoding, que cria nossos embeddings posicionais.

Esses embeddings posicionais funcionam de maneira análoga aos embeddings de tokens, mas em vez de codificar o significado da palavra, codificam sua posição na sequência. O processo final envolve somar os dois embeddings: o do token (que carrega o significado) e o posicional (que indica a posição). Esta abordagem permite que os Transformers processem as entradas em paralelo enquanto preservam a compreensão da estrutura sequencial do texto.

Existem diferentes abordagens para implementar o positional encoding. A técnica original, apresentada no artigo "Attention is All You Need", utiliza funções trigonométricas (seno e cosseno) para gerar os embeddings posicionais de forma determinística. Esta solução tem a vantagem de não adicionar parâmetros treináveis ao modelo.

Uma alternativa é utilizar uma camada adicional de embedding que aprende as representações posicionais durante o treinamento. Embora esta abordagem adicione parâmetros ao modelo, ela foi adotada com sucesso em arquiteturas posteriores como BERT e GPT-2, demonstrando resultados equivalentes à solução trigonométrica.

![positional_encoding](../assets/3-EMBEDDING%20POSICIONAL.png)

Para nosso modelo, usaremos os embeddings posicionais treináveis - uma camada adicional que aprende a representar a posição de cada token (0, 1, 2...) durante o treinamento. Isso difere da camada principal de embeddings, que trabalha com os IDs das palavras do vocabulário.

## **Implementando nossa solução**

Para criarmos a camada de embedding posicional do nosso modelo, podemos utilizar a função disponível na biblioteca Pytorch. Diferentemente da criação de embeddings convencionais, em vez de especificarmos o tamanho do vocabulário, devemos informar a quantidade máxima de tokens que o modelo será capaz de processar, conhecida como **janela de contexto.**

In [109]:
CONTEXT_LENGTH =  20# Número máximo de tokens que o modelo pode considerar (Janela de contexto)

# Criando a camada de embedding para posições
pos_embedding_layer = torch.nn.Embedding(CONTEXT_LENGTH, EMBEDDING_DIM)

# Recuperando os embeddings posicionais para cada token
pos_embeddings = pos_embedding_layer(torch.arange(len(tokens)))

# Somando os embeddings token e posicional para obter o embedding final
embedding = vectors + pos_embeddings

print("Frase:", phrase)
print("--- Embedding Token 1:\n", vectors[0])
print("Shape:", vectors.size())
print("--- Embedding Posicional 1:\n", pos_embeddings[0])
print("Shape:", pos_embeddings.size())
print("--- Embedding Final 1:\n", embedding[0])
print("Shape:", embedding.size())

Frase: O rato roeu a roupa do Rei de Roma
--- Embedding Token 1:
 tensor([-0.3925, -1.4036, -0.7279, -0.5594, -0.7688,  0.7624,  1.6423, -0.1596,
        -0.4974,  0.4396], grad_fn=<SelectBackward0>)
Shape: torch.Size([9, 10])
--- Embedding Posicional 1:
 tensor([ 1.3541, -0.0709,  2.1726,  2.2334,  0.2225, -0.9300,  0.3208,  0.0371,
         0.5675,  0.8284], grad_fn=<SelectBackward0>)
Shape: torch.Size([9, 10])
--- Embedding Final 1:
 tensor([ 0.9617, -1.4745,  1.4447,  1.6740, -0.5463, -0.1676,  1.9632, -0.1225,
         0.0701,  1.2680], grad_fn=<SelectBackward0>)
Shape: torch.Size([9, 10])


No código demonstrado, observamos como os embeddings finais são criados através de um processo de soma entre dois componentes: os embeddings dos tokens e os embeddings posicionais. Esta operação matemática permite que cada token mantenha sua representação semântica original enquanto incorpora informações sobre sua localização na sequência.

Durante este processo, o modelo mantém a representação original de cada token através do embedding de token, enquanto adiciona uma camada extra de informação posicional através do embedding posicional. Esta combinação resulta em um tensor que carrega tanto o significado quanto a posição da palavra no texto.

Esta abordagem permite que o modelo processe simultaneamente a semântica e a estrutura da frase, fornecendo uma base sólida para o aprendizado do nosso LLM.

## **Conclusão**

Os embeddings representam um pilar fundamental no processamento de linguagem natural, transformando texto em representações matemáticas precisas que permitem aos modelos de aprendizado de máquina compreender e processar a linguagem humana. Através dessas estruturas matemáticas, os modelos desenvolvem uma compreensão sofisticada de nuances linguísticas e relações contextuais.

A integração de embeddings com mecanismos avançados, especialmente os posicionais, impulsiona o desempenho excepcional dos modelos de linguagem modernos. Esta combinação permite que eles compreendam não apenas o significado das palavras, mas também sua posição e relacionamento dentro do texto.

Em nossa próxima discussão, mergulharemos nos **mecanismos de atenção** - um componente revolucionário dos Transformers que trabalha em harmonia com os embeddings. Exploraremos como esses mecanismos permitem que os modelos focalizem dinamicamente nas partes mais relevantes do texto, possibilitando uma compreensão profunda e uma geração de texto mais coerente.